In [3]:
#pip install lxml

     |████████████████████████████████| 5.5MB 5.8MB/s eta 0:00:01     |█████▌                          | 952kB 5.8MB/s eta 0:00:01�██████████████████████▍    | 4.7MB 5.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


#### This will grab data table from wikipedia website and place it in a pandas dataframe

In [19]:
import pandas as pd
myData1 =  pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = myData1[0]
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [21]:
df.shape

(180, 3)

#### This code will filter out all the boroughs labeled Not assigned and save to a new dataframe
#### #Probably not best idea for huge datasets...

In [20]:
my_df = df[df['Borough'] != 'Not assigned']
my_df.tail(10)

,Postal Code,Borough,Neighbourhood
151,M8W,Etobicoke,"Alderwood, Long Branch"
152,M9W,Etobicoke,"Northwest, West Humber - Clairville"
153,M1X,Scarborough,Upper Rouge
156,M4X,Downtown Toronto,"St. James Town, Cabbagetown"
157,M5X,Downtown Toronto,"First Canadian Place, Underground city"
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [12]:
my_df.shape

(103, 3)